<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/RRS_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install -q mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [3]:
!pip install -U -q tensorboard_plugin_profile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.3 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
from typing import Dict, Text

import pandas as pd
import tensorflow as tf

import tensorflow_recommenders as tfrs

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import IntegerLookup

import mlflow

# Recipe Recommendation System
## part 3 - Experiments

In [6]:
ratings_data = pd.read_csv("/content/gdrive/MyDrive/RRS_data/core-data-train_rating.csv")
ratings_data_val = pd.read_csv("/content/gdrive/MyDrive/RRS_data/core-data-valid_rating.csv")
recipes_data = pd.read_csv("/content/gdrive/MyDrive/RRS_data/core-data_recipe.csv")

In [7]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
ratings_val = tf.data.Dataset.from_tensor_slices(dict(ratings_data_val))
recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [8]:
ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"]
})

ratings_val = ratings_val.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"]
})

recipes = recipes.map(lambda x: x["recipe_id"])

In [9]:
for r in ratings.take(1):
  print(type(r))
  for k, v in r.items():
    print(f"{k}: {v}")

<class 'dict'>
recipe_id: 17991
user_id: 5215572


In [10]:
for r in recipes.take(1):
  print(r)

tf.Tensor(240488, shape=(), dtype=int64)


In [11]:
ratings.cardinality(), recipes.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=676946>,
 <tf.Tensor: shape=(), dtype=int64, numpy=45630>)

In [12]:
unique_user_ids = ratings_data.user_id.unique()
len(unique_user_ids)

68768

In [13]:
unique_recipe_ids = recipes_data.recipe_id.unique()
len(unique_recipe_ids)

45630

In [14]:
embedding_dimension = 32

In [15]:
# query tower

user_model = Sequential([
  IntegerLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [16]:
# candidate tower
recipe_model = Sequential([
  IntegerLookup(
      vocabulary=unique_recipe_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
])

In [17]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=recipes.batch(128).map(recipe_model)
)

In [18]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [19]:
class BaseRetrievalModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.recipe_model: tf.keras.Model = recipe_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_recipe_embeddings = self.recipe_model(features["recipe_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_recipe_embeddings)

In [20]:
model = BaseRetrievalModel(user_model, recipe_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [21]:
ratings_train = ratings.batch(512)
ratings_val = ratings_val.batch(512)

In [22]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = "/content/log")

In [23]:
experiment = mlflow.create_experiment("base_retrieval")

In [24]:
mlflow.autolog()

2024/01/25 11:09:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [ ]:
with mlflow.start_run(experiment_id = experiment, run_name="autolog_01") as run:
  model.fit(ratings_train, validation_data= ratings_val, epochs = 10, callbacks = [tensorboard_callback])

2024/01/25 11:09:30 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: The layer "base_retrieval_model" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.
2024/01/25 11:09:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/mlflow/tensorflow/_autolog.py:52: UserWarning: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data."


Epoch 1/10
1323/1323 [==============================] - 2491s 2s/step - factorized_top_k/top_1_categorical_accuracy: 1.7727e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0202 - factorized_top_k/top_100_categorical_accuracy: 0.0323 - loss: 3189.7506 - regularization_loss: 0.0000e+00 - total_loss: 3189.7506 - val_factorized_top_k/top_1_categorical_accuracy: 3.6715e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0027 - val_factorized_top_k/top_10_categorical_accuracy: 0.0050 - val_factorized_top_k/top_50_categorical_accuracy: 0.0191 - val_factorized_top_k/top_100_categorical_accuracy: 0.0315 - val_loss: 1974.8689 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1974.8689
Epoch 2/10
1323/1323 [==============================] - 2453s 2s/step - factorized_top_k/top_1_categorical_accuracy: 2.5408e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0520 - factor

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

# %reload_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/log"